In [1]:
import xgboost as xgb
print(xgb.__version__)

3.1.1


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

c:\Users\Ar\Desktop\Regression_ML_EndtoEnd\.venv\Scripts\python.exe
3.1.1
c:\Users\Ar\Desktop\Regression_ML_EndtoEnd\.venv\Lib\site-packages\xgboost\__init__.py


In [3]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

c:\Users\Ar\Desktop\Regression_ML_EndtoEnd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Ar\Desktop\Regression_ML_EndtoEnd\.venv\Lib\site-packages\mlflow\protos\service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


In [7]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("/Users/Ar/Desktop/Regression_ML_EndtoEnd/data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("/Users/Ar/Desktop/Regression_ML_EndtoEnd/data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (302953, 39)
Eval shape: (148448, 39)


In [ ]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)    
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [15]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("file:///C:/Users/Ar/Desktop/Regression_ML_EndtoEnd/mlruns")

mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/11/04 23:11:23 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-11-04 23:11:23,457] A new study created in memory with name: no-name-b04d5c04-bbbe-4caf-8bc3-181d80846d7f
[I 2025-11-04 23:13:24,638] Trial 0 finished with value: 100605.9061962163 and parameters: {'n_estimators': 931, 'max_depth': 9, 'learning_rate': 0.17568229184446024, 'subsample': 0.8563153160639544, 'colsample_bytree': 0.9142267767757526, 'min_child_weight': 4, 'gamma': 3.7707564907380298, 'reg_alpha': 1.8754760264793737e-08, 'reg_lambda': 0.015934915034022556}. Best is trial 0 with value: 100605.9061962163.
[I 2025-11-04 23:14:15,057] Trial 1 finished with value: 102252.87983005437 and parameters: {'n_estimators': 574, 'max_depth': 8, 'learning_rate': 0.20713470465283648, 'subsample': 0.9854102148212645, 'colsample_bytree': 0.8127908519252744, 'min_child_weight': 6, 'gamma': 2.4235966760996757, 'reg_alpha': 0.005566975691283893, 'reg_la

Best params: {'n_estimators': 897, 'max_depth': 6, 'learning_rate': 0.017793994665602813, 'subsample': 0.6064593751565328, 'colsample_bytree': 0.7957593918588906, 'min_child_weight': 3, 'gamma': 4.476373287434135, 'reg_alpha': 9.626395373005773e-07, 'reg_lambda': 9.066931890408167e-07}


In [ ]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)    

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, artifact_path="model")
